In [5]:
import numpy as np
import yfinance as yf
import pandas as pd
from math import sqrt
import tasa_vasicek as tv
from numpy.linalg import inv 

In [7]:
empresas=pd.read_excel('data_final.xlsx')

In [8]:
symbols=empresas['symbol']

In [84]:
class portfolio:
    def __init__(self,symbols):
        self.symbs=sorted(symbols)
    @property
    def Close(self):
        data=yf.download(tickers=self.symbs, start='2019-6-1',progress=0).Close
        data=data.dropna(axis=0)
        return data   
    @property
    def matriz_rendimientos(self):
        def rendimiento(comp): return np.diff(np.log(comp)) # return [log(comp[i]/comp[i-1]) for i in range(1,len(comp))]
        df_rend=self.Close.apply(rendimiento,axis=0)
        return df_rend
    @property
    def rend_volt_anual(self):
        rendimientos=self.matriz_rendimientos
        medias=list(rendimientos.mean()*252)
        varianzas=list(rendimientos.std()*sqrt(len(rendimientos)*252/(len(rendimientos)+1)))
        return medias,varianzas
    @property
    def MVC(self):
        medias,varianzas=self.rend_volt_anual
        Matriz_Si=[[0]*len(varianzas) for i in range(len(varianzas))]
        for i,s_i in zip(range(len(Matriz_Si)),range(len(varianzas))):
                Matriz_Si[i][s_i]=varianzas[s_i]
        Matriz_Si=np.array(Matriz_Si)
        Matrizcorr=self.Close.corr()
        Matrizcorr=np.array(Matrizcorr)
        return Matriz_Si@Matrizcorr@Matriz_Si.T
    @property
    def portafolios(self):
        n=len(self.symbs)
        mvc=self.MVC
        Eri=self.rend_volt_anual[0]
        Vector_B=np.array([0]*n+[1]).reshape(n+1,1)
        def sigmaW(Port): return sqrt(Port@mvc@Port.T)
        def rendW(Porc_w): return sum([p*r for p,r in zip(Eri,Porc_w)])
        mvca=np.array([list(x)+[1] for x in mvc]+[[1]*n+[0]])
        mvca_inv=inv(mvca)
        w=mvca_inv@Vector_B
        w=np.abs(w[:-1])
        w=w/sum(w)
        return rendW(w)

In [85]:
my_portfolio=portfolio(list(symbols[:20]))

In [86]:
my_portfolio.portafolios

array([0.9549903])

In [8]:
mvc=my_portfolio.MVC

In [10]:
Vector_B=np.array([0]*20+[1]).reshape(21,1)


In [19]:
x=np.array([[1,2],[3,4]])
np.append(x,[5,6]).reshape(3,2)

array([[1, 2],
       [3, 4],
       [5, 6]])

In [29]:
mvca=np.array([list(x)+[1] for x in mvc]+[[1]*20+[0]])

In [30]:
mvca_inv=np.linalg.inv(mvca)

In [31]:
W_min_riesgo=mvca_inv@Vector_B


In [50]:
p=W_min_riesgo[:-1]

In [54]:
p=p.reshape(1,20)

In [43]:
e=my_portfolio.rend_volt_anual[0]

In [38]:
l=np.abs(p)/sum(np.abs(p))

In [68]:
l=l.reshape(1,20)[0]

In [69]:
l

array([0.01310811, 0.02136327, 0.07505094, 0.10066247, 0.02120561,
       0.11329923, 0.05255793, 0.08431731, 0.07166539, 0.02363396,
       0.17170028, 0.0566971 , 0.0563375 , 0.01314865, 0.03095874,
       0.01980955, 0.00646578, 0.02285899, 0.01403581, 0.03112339])

In [70]:
sum(p[0]*np.array(e)),sum(l*np.array(e))


(-0.07836678693113862, 0.16375192856999735)

In [58]:
# ,sum((np.array(e)*np.array(l)))

array([ 0.02645992,  0.00298319, -0.03996887,  0.0775007 ,  0.01723447,
       -0.1163991 , -0.02379139,  0.03316278, -0.03895454, -0.01608262,
        0.20155748, -0.14819788,  0.02998049, -0.0044507 , -0.14010715,
        0.02598806,  0.00834786,  0.01076771, -0.00925961,  0.02486243])

In [39]:
sum(l)

array([1.])

In [15]:
df_portafolios=my_portfolio.portafolios

In [16]:
df_portafolios[df_portafolios['Clasificación']=='b'].sort_values('Volatilidad')

,Portafolio,Clasificación,Rendimiento,Volatilidad
7497,"[0.01636449215493735, 0.03462611702561387, 0.0...",b,0.111346,0.215590
1600,"[0.0924074250417687, 0.008059475143097844, 0.0...",b,0.126666,0.216290
2798,"[0.06876170455923271, 0.006205635821710601, 0....",b,0.123257,0.216403
1270,"[0.033264306336596584, 0.03833427135181946, 0....",b,0.106618,0.216695
1019,"[0.03207209656760786, 0.011656040961225331, 0....",b,0.117701,0.217149
...,...,...,...,...
7513,"[0.02281759007933998, 0.0587900200392756, 0.02...",b,0.174667,0.251886
6939,"[0.09073378641334946, 0.05545004861983039, 0.0...",b,0.162787,0.251897
576,"[0.07584955267884737, 0.04649447879564567, 0.0...",b,0.157958,0.251899
9742,"[0.029616169421359428, 0.06157189762476549, 0....",b,0.161270,0.251907


In [17]:
df_portafolios[df_portafolios['Clasificación']=='m'].sort_values('Volatilidad')

,Portafolio,Clasificación,Rendimiento,Volatilidad
3085,"[0.00028031359902935983, 0.09121857599271713, ...",m,0.148172,0.251912
3676,"[0.04622630488868083, 0.07367644008334372, 0.0...",m,0.159496,0.251922
2509,"[0.015996279216208976, 0.09870834995837743, 0....",m,0.145955,0.251922
6623,"[0.06283972547557835, 0.0748429343767748, 0.08...",m,0.158915,0.251931
5746,"[0.02272565396198686, 0.0788810150554787, 0.08...",m,0.118779,0.251937
...,...,...,...,...
6181,"[0.010067680744654322, 0.04813185301234674, 0....",m,0.152333,0.262744
9814,"[0.05068916019058499, 0.10276402359459143, 0.0...",m,0.160571,0.262748
3491,"[0.04758143430557668, 0.07156718824309267, 0.0...",m,0.154565,0.262753
2423,"[0.03603075026196364, 0.05860980964725143, 0.0...",m,0.156975,0.262755


In [18]:
df_portafolios[df_portafolios['Clasificación']=='a'].sort_values('Volatilidad')

,Portafolio,Clasificación,Rendimiento,Volatilidad
5464,"[0.04162764664885956, 0.05859972759456969, 0.0...",a,0.162135,0.262761
9534,"[0.038324169439799535, 0.029347455468162868, 0...",a,0.159075,0.262768
9827,"[0.06670463919390754, 0.07614846849608176, 0.0...",a,0.168613,0.262771
7441,"[0.02081029449913557, 0.057042047968330234, 0....",a,0.118351,0.262775
8780,"[0.008939673507268328, 2.38085999157185e-06, 0...",a,0.166422,0.262796
...,...,...,...,...
5626,"[0.08108878691206392, 0.003236659360369082, 0....",a,0.212012,0.316541
9003,"[0.07646390456045297, 0.040982878273216894, 0....",a,0.200602,0.317031
2639,"[0.010552448213359482, 0.012692977688260752, 0...",a,0.216440,0.317336
3181,"[0.11050772995186438, 0.07202387067047243, 0.0...",a,0.239632,0.321420


In [37]:
cetes_28=tv.cetes_28.iloc[-1]

In [45]:
port_ejemplo=df_portafolios.iloc[144]['Portafolio']

In [35]:
rend_empresas=np.array(my_portfolio.rend_volt_anual[0])
rend_empresas

array([ 0.35210708,  0.02605035,  0.08954605,  0.12752321,  0.13967634,
        0.17927431,  0.077796  ,  0.06672485,  0.09341664, -0.11429261,
        0.2012678 ,  0.45274816,  0.09145696, -0.05817324,  0.77849654,
        0.22998282,  0.22188595,  0.07469058,  0.11328145,  0.13532874])

In [71]:
rend_exceso=rend_empresas*port_ejemplo-np.mean(tv.cetes_28.iloc[-30])

In [72]:
rend_exceso

array([-0.03844737, -0.05321082, -0.0521883 , -0.05004089, -0.0545486 ,
       -0.04647782, -0.04995571, -0.05461853, -0.04618915, -0.06769116,
       -0.0309303 , -0.00172207, -0.05394445, -0.05520863,  0.04285948,
       -0.05180078, -0.05420744, -0.04606694, -0.05466153, -0.05334935])

In [66]:
Rend_Exces=rend_exceso
MVC=my_portfolio.MVC
V_vector=np.linalg.inv(MVC)@Rend_Exces
W_tan=V_vector/V_vector.sum()


In [67]:
W_tan

array([ 0.02487349,  0.31751578, -0.23586759,  0.07562907,  0.30898086,
       -0.07126182, -0.27532105,  0.35362079, -0.20260976,  0.10315465,
        0.36517176, -0.36732309,  0.35839716, -0.02435218, -0.36990012,
        0.43560569,  0.455875  , -0.21126524, -0.03484224, -0.00608115])

In [54]:
sum(W_tan>0)

11

In [74]:
np.mean(tv.cetes_28.iloc[-31])

0.060899999999999996

In [2]:
import numpy as np

In [3]:
import yfinance as yf

In [13]:
simbolos=list(symbols[:15])

In [40]:
pf_example=np.random.random(15)
pf_example=pf_example/sum(pf_example)
sum(pf_example)

1.0

In [53]:
monto_incial=1_000
monto=monto_incial*lst_prices
acciones=[int(a) for a in monto/lst_prices]
acciones

[1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000]

In [54]:
matrix=np.array([[2,7,1],[9,3,5],[10,6,2]]).reshape(3,3)

In [56]:
np.linalg.det(matrix)

200.0000000000001